# 🚀 ELS GPU 테스트 - 수정 버전

**Google Drive 방식 (버그 수정)**

---

## ⚡ Step 0: GPU 활성화

1. `Runtime` → `Change runtime type`
2. Hardware accelerator: **GPU**
3. Save


## 📁 Step 1: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive 연결 완료!")

## 📤 Step 2: 프로젝트 파일 업로드

**로컬 컴퓨터에서:**

1. Google Drive 열기 (drive.google.com)
2. `els-fdm-pricer-colab.tar.gz` 파일을 드래그 앤 드롭으로 업로드
3. 업로드 완료 후 아래 셀 실행

In [ ]:
# Drive에서 파일 복사 및 압축 해제
!cp /content/drive/MyDrive/els-fdm-pricer-colab.tar.gz .
!tar -xzf els-fdm-pricer-colab.tar.gz

print("✓ 프로젝트 파일 준비 완료!")

## 🔧 Step 3: 패키지 설치

In [ ]:
# CuPy 설치
!pip install -q cupy-cuda12x numpy scipy matplotlib

print("✓ 패키지 설치 완료!")

## 🎮 Step 4: GPU 확인

In [ ]:
# nvidia-smi로 GPU 정보 확인
!nvidia-smi

# CuPy로 GPU 확인 (수정된 방법)
import cupy as cp

print("\n" + "="*80)
print("GPU 정보")
print("="*80)

if cp.cuda.is_available():
    # GPU 이름 가져오기 (올바른 방법)
    device_id = 0
    props = cp.cuda.runtime.getDeviceProperties(device_id)
    gpu_name = props['name'].decode('utf-8')
    
    print(f"✓ GPU 사용 가능: True")
    print(f"✓ GPU 이름: {gpu_name}")
    
    # 메모리 정보
    mem_info = cp.cuda.Device(device_id).mem_info
    total_mem = mem_info[1] / 1024**3
    free_mem = mem_info[0] / 1024**3
    
    print(f"✓ GPU 메모리 (총): {total_mem:.1f} GB")
    print(f"✓ GPU 메모리 (사용가능): {free_mem:.1f} GB")
    
    # Compute capability
    compute_capability = cp.cuda.Device(device_id).compute_capability
    print(f"✓ Compute Capability: {compute_capability}")
else:
    print("⚠️ GPU를 사용할 수 없습니다.")
    print("Runtime → Change runtime type → GPU로 설정했는지 확인하세요!")

print("="*80)

## 🧪 Step 5: GPU 테스트

In [ ]:
import sys
sys.path.insert(0, '/content/els-fdm-pricer')

from src.models.els_product import create_sample_els
from src.pricing.gpu_els_pricer import price_els_gpu
from src.pricing.els_pricer import price_els
import time

print("="*80)
print("GPU vs CPU 벤치마크")
print("="*80)

product = create_sample_els()

# CPU 테스트
print("\n[CPU] 100×100×200 계산 중...", end="", flush=True)
start = time.time()
result_cpu = price_els(product, N1=100, N2=100, Nt=200, verbose=False)
time_cpu = time.time() - start
print(f" {time_cpu:.2f}초")

# GPU 테스트
print("[GPU] 100×100×200 계산 중...", end="", flush=True)
start = time.time()
result_gpu = price_els_gpu(product, N1=100, N2=100, Nt=200, use_gpu=True, verbose=False)
time_gpu = time.time() - start
print(f" {time_gpu:.2f}초")

speedup = time_cpu / time_gpu

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"CPU 시간: {time_cpu:.2f}초")
print(f"GPU 시간: {time_gpu:.2f}초")
print(f"속도 향상: {speedup:.1f}배")
print(f"가격 (CPU): {result_cpu['price']:.4f}")
print(f"가격 (GPU): {result_gpu['price']:.4f}")
print(f"가격 차이: {abs(result_cpu['price'] - result_gpu['price']):.6f}")
print("="*80)

if speedup > 20:
    print("\n🚀 GPU 매우 빠름! (20배 이상)")
elif speedup > 10:
    print("\n✓ GPU 상당히 빠름! (10배 이상)")
else:
    print("\n✓ GPU 효과 확인!")

## 🎯 Step 6: 대규모 테스트 (200×200×1000)

In [ ]:
print("="*80)
print("대규모 그리드 GPU 테스트 (200×200×1000)")
print("="*80)
print()
print("총 계산량: 40,000,000 포인트")
print("예상 시간: 1-2초 (GPU)")
print()

print("계산 중...", end="", flush=True)
start = time.time()
result = price_els_gpu(product, N1=200, N2=200, Nt=1000, use_gpu=True, verbose=False)
elapsed = time.time() - start
print(" 완료!")

print("\n" + "="*80)
print("결과")
print("="*80)
print(f"ELS 가격: {result['price']:.4f}")
print(f"GPU 시간: {elapsed:.2f}초")
print(f"처리량: {40_000_000/elapsed:,.0f} points/sec")
print()
print("CPU 시간 (측정값): 78.26초")
speedup_large = 78.26 / elapsed
print(f"속도 향상: {speedup_large:.1f}배")
print("="*80)

if elapsed < 2.0:
    print("\n🎉 실시간 프라이싱 가능! (< 2초)")
elif elapsed < 5.0:
    print("\n✓ 준실시간 프라이싱 가능! (< 5초)")
else:
    print("\n△ 추가 최적화 권장")

## 📊 Step 7: 추가 벤치마크 (다양한 그리드)

In [ ]:
import numpy as np

print("="*80)
print("다양한 그리드 크기 벤치마크")
print("="*80)
print()

test_configs = [
    (80, 80, 160),
    (100, 100, 200),
    (150, 150, 300),
    (200, 200, 400)
]

print(f"{'Grid Size':<15} {'CPU Time':<12} {'GPU Time':<12} {'Speedup':<12}")
print("-" * 80)

results = []

for N1, N2, Nt in test_configs:
    print(f"{N1}×{N2}×{Nt:<5}", end="", flush=True)
    
    # CPU
    start = time.time()
    r_cpu = price_els(product, N1=N1, N2=N2, Nt=Nt, verbose=False)
    t_cpu = time.time() - start
    
    # GPU
    start = time.time()
    r_gpu = price_els_gpu(product, N1=N1, N2=N2, Nt=Nt, use_gpu=True, verbose=False)
    t_gpu = time.time() - start
    
    speedup = t_cpu / t_gpu
    results.append(speedup)
    
    print(f"  {t_cpu:>9.2f}s  {t_gpu:>9.2f}s  {speedup:>9.1f}x")

print("-" * 80)
print(f"평균 속도 향상: {np.mean(results):.1f}배")
print(f"최대 속도 향상: {np.max(results):.1f}배")
print("="*80)

## 📈 Step 8: 결과 시각화

In [ ]:
import matplotlib.pyplot as plt

# 그래프 그리기
grid_labels = [f"{N1}×{N2}×{Nt}" for N1, N2, Nt in test_configs]

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.bar(grid_labels, results, alpha=0.8, color='green')
ax.set_xlabel('Grid Size', fontsize=12)
ax.set_ylabel('Speedup (×)', fontsize=12)
ax.set_title('GPU Speedup over CPU', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
ax.axhline(y=1, color='r', linestyle='--', label='No speedup')

# 값 표시
for i, v in enumerate(results):
    ax.text(i, v + 1, f'{v:.1f}x', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("✓ 시각화 완료!")

## 🎉 최종 요약

In [ ]:
print("="*80)
print("🎉 GPU 성능 테스트 완료!")
print("="*80)
print()
print("GPU 정보:")
print(f"  - GPU: {gpu_name}")
print(f"  - 메모리: {total_mem:.1f} GB")
print()
print("성능 결과:")
print(f"  - 평균 속도 향상: {np.mean(results):.1f}배")
print(f"  - 최대 속도 향상: {np.max(results):.1f}배")
print(f"  - 대규모 그리드(200×200×1000): {speedup_large:.1f}배")
print()
print("결론:")
if np.mean(results) > 20:
    print("  🚀 GPU 사용 강력 권장! 실시간 프라이싱 가능!")
elif np.mean(results) > 10:
    print("  ✓ GPU 사용 권장! 성능 크게 향상됨!")
else:
    print("  △ GPU 효과 있음. 최적화로 더 개선 가능.")
print()
print("다음 단계:")
print("  1. GPU 최적화 구현 (Batched solver, Vectorized callbacks)")
print("  2. 예상 추가 향상: 10-15배")
print("  3. 최종 목표: < 0.1초 (실시간 프라이싱)")
print("="*80)